In [12]:
import pandas as pd
from datetime import datetime

In [13]:
import great_expectations as gx
from great_expectations.core.batch import RuntimeBatchRequest
from great_expectations.core.yaml_handler import YAMLHandler
from great_expectations.data_context.types.base import (
    DataContextConfig,
    InMemoryStoreBackendDefaults,
)
from great_expectations.core.expectation_configuration import ExpectationConfiguration
from great_expectations.core.expectation_suite import ExpectationSuite
from gx_demo_dataprep import trips_sample_random
yaml = YAMLHandler()

In [14]:
trips_df = trips_sample_random()

In [15]:
# INITIALIZING THE DATA CONTEXT
data_context_configuration = DataContextConfig(
    store_backend_defaults=InMemoryStoreBackendDefaults()
)

data_context = gx.get_context(project_config=data_context_configuration)

In [16]:
# CONFIGURE DATASOURCE

datasource_name = "nyc_yellowcab_trips"

datasource_config = {
    "name": datasource_name,
    "class_name": "Datasource",
    "execution_engine": {"class_name": "PandasExecutionEngine"},
    "data_connectors": {
        datasource_name: {
            "class_name": "RuntimeDataConnector",
            "batch_identifiers": ["batch_id", "batch_datetime"],
        }
    },
}

data_context.test_yaml_config(yaml.dump(datasource_config))
data_context.add_datasource(**datasource_config)

Attempting to instantiate class from config...
	Instantiating as a Datasource, since class_name is Datasource
	Successfully instantiated Datasource


ExecutionEngine class name: PandasExecutionEngine
Data Connectors:
	nyc_yellowcab_trips:RuntimeDataConnector

	Available data_asset_names (0 of 0):
		Note : RuntimeDataConnector will not have data_asset_names until they are passed in through RuntimeBatchRequest

	Unmatched data_references (0 of 0): []



In [17]:
# CREATE BATCH REQUEST
dataset_name = datasource_name

batch_request = RuntimeBatchRequest(
    datasource_name=datasource_name,
    data_connector_name=dataset_name,
    data_asset_name=dataset_name,
    runtime_parameters={"batch_data": trips_df},  # our nyc trips dataframe
    batch_identifiers={
        "batch_id": f"{dataset_name}_{datetime.strftime(datetime.now(), '%Y%m%d%H%M%S')}",
        
    },
)

In [18]:
# WAYS OF SETTING UP YOUR EXPECTATIONS AND EVALUATING THEM

#  1. Create an empty Expectation Suite and  and use an Expectation Validator to evaluate your expectations
#  2. Create a set of Expectations and evalutate them using a Checkpoint

In [19]:
# 2. Building a set of expectations and Evaluating them using a GX Checkpoint

expectation_suite_name = f"{dataset_name}_expectation_suite"

expectation_config_dicts = [
    {
        "expectation_type": "expect_column_values_to_not_be_null",
        "kwargs": {
            "column": "passenger_count"
        }
    },
    {
        "expectation_type": "expect_column_values_to_not_be_null",
        "kwargs": {
            "column": "trip_distance"
        }
    }
]

expectations: list = []
expectation_suite = ExpectationSuite(expectation_suite_name=expectation_suite_name)

for item in expectation_config_dicts:
    config = ExpectationConfiguration(**item)
    expectations.append(config)

expectation_suite.add_expectation_configurations(expectations)

data_context.add_or_update_expectation_suite(expectation_suite=expectation_suite)


{
  "expectations": [
    {
      "meta": {},
      "kwargs": {
        "column": "passenger_count"
      },
      "expectation_type": "expect_column_values_to_not_be_null"
    },
    {
      "meta": {},
      "kwargs": {
        "column": "trip_distance"
      },
      "expectation_type": "expect_column_values_to_not_be_null"
    }
  ],
  "meta": {
    "great_expectations_version": "0.16.10"
  },
  "data_asset_type": null,
  "expectation_suite_name": "nyc_yellowcab_trips_expectation_suite",
  "ge_cloud_id": null
}

In [20]:
# CREATING A CHECKPOINT TO VALIDATE EXPECTATIONS
checkpoint_name = f"{dataset_name}_checkpoint"

checkpoint_config = {
    "name": checkpoint_name,
    "config_version": 1.0,
    "class_name": "SimpleCheckpoint",
    "run_name_template": f"{dataset_name}_%Y%m%d-%H%M%S",
}

data_context.add_or_update_checkpoint(**checkpoint_config)

{
  "action_list": [
    {
      "name": "store_validation_result",
      "action": {
        "class_name": "StoreValidationResultAction"
      }
    },
    {
      "name": "store_evaluation_params",
      "action": {
        "class_name": "StoreEvaluationParametersAction"
      }
    },
    {
      "name": "update_data_docs",
      "action": {
        "class_name": "UpdateDataDocsAction",
        "site_names": []
      }
    }
  ],
  "batch_request": {},
  "class_name": "Checkpoint",
  "config_version": 1.0,
  "evaluation_parameters": {},
  "module_name": "great_expectations.checkpoint",
  "name": "nyc_yellowcab_trips_checkpoint",
  "profilers": [],
  "run_name_template": "nyc_yellowcab_trips_%Y%m%d-%H%M%S",
  "runtime_configuration": {},
  "validations": []
}

In [21]:
# VALIDATING RESULTS USING CHECKPOINT
checkpoint_result = data_context.run_checkpoint(
    checkpoint_name=checkpoint_name,
    validations=[
        {
            "batch_request": batch_request,
            "expectation_suite_name": expectation_suite_name
        }
    ]
)

Calculating Metrics:   0%|          | 0/13 [00:00<?, ?it/s]

In [22]:
assert(checkpoint_result['success']==True)

AssertionError: 